In [6]:
import sys

sys.path.append('..')
import os
os.getcwd()
os.chdir('..')

import pandas as pd
pd.set_option('display.max_rows' , 500)
pd.set_option('display.max_columns' , 200)

from sample.data_prep.data_extract import Dataset
from sample.method.other_function import format_sql_query
from sample.data_prep.data_pre_tests import metric_test

In [7]:
var_dict = {0:'AA' , 1:'A' , 2:'B' , 3:'C' , 4:'D' , 5:'E'}
def style_row(row):
    if row.name % 2 == 0:
        if row.name ==4:
            return ['background-color: lavender ; color:firebrick;font-weight: bold' for _ in row]
        # 偶数行设置为深蓝色
        return ['background-color: lavender ; color:black' for _ in row]
    else:
        # 奇数行设置为浅蓝色
        return ['background-color: ghostwhite;color:black' for _ in row]




def exp_results(df,column ,test_var,test_type = 0 , device = 'all',var_name = var_dict , merge_aa = True):
    name_dict = {f'all_{column}':column ,'index':'',f'device_{column}':column}
    
    if device == 'all':
        df = df.rename({column:f'all_{column}'} , axis =1)
        temp = df.groupby(['var_id' , 'beacon_uid'])[f'all_{column}'].mean().groupby('var_id').mean()
    else:
        df = df.rename({column:f'device_{column}'} , axis =1)
        temp = df.query('device == @device').groupby(['var_id' , 'beacon_uid'])[f'device_{column}'].mean().groupby('var_id').mean()

    df_metric = pd.DataFrame(metric_test(df ,test_var , device = device ,metrics_list = [column], significant_level = .05 , merge_aa = merge_aa))
    df_metric.loc['mean_diff%'] = df_metric.loc['mean_diff']/df_metric.loc['group_1_mean']
    df_metric = df_metric.loc[['group_1_mean' , 'group_2_mean' ,'mean_diff', 'mean_diff%','is_significant','p_value']].rename({'group_1_mean':f'var_{var_name[test_var[0]]}','group_2_mean':f'var_{var_name[test_var[1]]}'} )
    
    if test_type == 0:
        display(pd.concat([df_metric.iloc[:4].applymap(lambda x:'{:.1%}'.format(x)) , df_metric.iloc[4:]]).reset_index().rename(name_dict,axis =1).style.apply(style_row, axis=1))
    else:
        display(pd.concat([df_metric.iloc[:3].applymap(lambda x:'{:.2f}'.format(x)),df_metric.iloc[3].to_frame().T.applymap(lambda x:'{:.1%}'.format(x)) , df_metric.iloc[4:]] , axis = 0).reset_index().rename(name_dict,axis =1).style.apply(style_row, axis=1))
    
    print(device)
    return temp
    
    

# Experiment Input

In [4]:
# exp_input = {
#     'exp_id':'beq_in_shop_search_abexp', 
#     'start':'2023-7-25',
#     'end':'2023-8-6',
#     'device':['iphone' ,'android' , 'web'], 
#     'version':'6.16.0' ,
#     'geo':['TW'],
#     'lang':['zh_TW'],
#     'trigger':{
#         # dict - key:str=event_name, value:tuple=event_parameter_filter
#         # 用 OR 連接
#         'view_item_recommend': None,
#         'view_item':(
#             # tuple - event_parameter_filter
#             # 用 OR 連接
#             {
#                 # dict - key:str=event_parameter, value:str/int/double=event_parameter_value
#                 # 用 AND 連接
#                 'from_screen':{
#                     'cal':'in',
#                     'value':['item']
#                 }
#             },
#             {
#                 'from_section':{
#                     'cal':'in',
#                     'value':['bottom_similar_item' , 'side_similar_item']
#                 }
#             }
#         ),
#         'impression_modal':{
#             'modal':{
#                 'cal':'=',
#                 'value':'bottom_sheet'
#                 },
#             'position':{
#                 'cal':'>=',
#                 'value':1
#             }
#         }
#     }
# }

exp_input = {
    'exp_id':'beq_for_you_abexp', 
    'start':'2023-9-13',
    'end':'2023-10-1',
    'device':['iphone' ,'android'], 
    'version':'6.23.0',
    'trigger':{'view_feed': None}
}

# Initialize Dataset
data_ = Dataset(**exp_input)

# Raw Data

In [3]:
# 1. raw_data (fro SRM 要 test 的指標)
sql_query = data_.get_raw_data(mode='sql')
# print(sql_query)
raw_data = data_.get_raw_data(mode='data')
print(raw_data.shape)
raw_data.head()

(35413, 10)


,exp_id,var_id,beacon_uid,device,device_avg_view_item,device_avg_duration,device_e2,all_avg_view_item,all_avg_duration,all_e2
0,beq_for_you_abexp,3,fb_906933389371787,iphone,14.0,3803.000000,True,14.0,3803.000000,True
1,beq_for_you_abexp,1,fb_1815269672075341,android,9.0,548.333333,True,9.0,548.333333,True
2,beq_for_you_abexp,3,armp4,iphone,7.0,763.000000,True,7.0,763.000000,True
3,beq_for_you_abexp,3,fb_166339073730371,iphone,13.0,1282.000000,True,13.0,1282.000000,True
4,beq_for_you_abexp,2,ll_plyncne2kzsvocsyb5nwz22uue,android,58.0,1291.125000,True,58.0,1291.125000,True


### SRM Check

In [4]:
from sample.data_prep.data_pre_tests import * 
srm_result = srm_check(
    df=raw_data,
    check_var_id_list=[0,1],
    ratio_list=[25,25],
    device='all',
    significant_level=0.01
)
srm_result

{'all': {'chi2_stat': 0.1074404761904762,
  'p_value': 0.7430771478039213,
  'is_significant': False}}

### AA Test

In [5]:
aa_result = metric_test(
    df=raw_data,
    check_var_id_list=['0', '1'],
    device='all',
    significant_level=0.01
)
aa_result

{'all_avg_view_item': {'group_1_mean': 30.37171687193946,
  'group_2_mean': 30.27518280853604,
  'mean_diff': -0.09653406340341775,
  'z_stat': -0.08549457119822539,
  'p_value': 0.9318682112273644,
  'is_significant': False,
  'se': 1.1291250666617942},
 'all_avg_duration': {'group_1_mean': 927.0380108954514,
  'group_2_mean': 902.1022055908415,
  'mean_diff': -24.935805304609858,
  'z_stat': -1.212689603406725,
  'p_value': 0.22524851622846676,
  'is_significant': False,
  'se': 20.56239719921687},
 'all_e2': {'group_1_mean': 0.7852797150912598,
  'group_2_mean': 0.7789882107148187,
  'mean_diff': -0.006291504376441104,
  'z_stat': -0.8834072601205695,
  'p_value': 0.3770162695599173,
  'is_significant': False,
  'se': 0.00712186175103703}}

# Exp Data

In [6]:
# 2. exp_data
sql_query = data_.get_exp_data(mode='sql')
# print(sql_query)
exp_data = data_.get_exp_data(mode='data')
print(exp_data.shape)
exp_data.head()

(284509, 6)


,beacon_uid,uid,beacon,exp_id,var_id,created_date
0,20230830ghf7m9hT6a,,20230830ghf7m9hT6a,beq_for_you_abexp,2,2023-09-24
1,fb_100000083483010,fb_100000083483010,20221014JY6zzDYUHS,beq_for_you_abexp,3,2023-09-22
2,gg_107507614741769590292,gg_107507614741769590292,20230830m6CfKme5LZ,beq_for_you_abexp,0,2023-09-17
3,kmimi5025,kmimi5025,20230830myzgH4rXDs,beq_for_you_abexp,3,2023-09-16
4,fb_819583144883654,fb_819583144883654,20221014MkaFtrjnbB,beq_for_you_abexp,3,2023-09-20


# Metric Data

metric 用 dict 紀錄，裡面主要包含兩大塊 metric_trigger, metric_main，一次只能算一個 metric，可以有多個 metric trigger

metric_trigger 是紀錄 metric 額外的 trigger ，會限制計算的 base 

    可以用 tuple 包不同 source 的 trigger, e.g. adoption e1->e2 trigger adoption,e1
    {metric_trigger:({source:'journey' , param:{funnel_e1:None}} , {source:'tracking' , param:{trigger_map}})

metric_main 是 metric 的計算邏輯，裡面主要包含 metirc 以及 condition

    condition 可以視為 metric 的 where 條件，是 metric 的計算 base e.g. measure_dwell_time 需要 condition 設置 view_item
    metirc 是計算正式的 param e.g. measure_dwell_time 的 metric 設置為 current_timer, view_item_cnt 則 metric 是 view_id
    main 裡如果是 boolean 可以只有 condition，但如果 journey metric 的部分特別會放在 metric 中
    metric_main:{metric_name:xxx , 'source':'tracking' , group_level':'event' ,event_type = 'event_on_screen', method:'boolean/count_distinct',  condition:{} , metric:{}}}

source: 取用的 data source

group_level: 後續要 group 的層級，user, day, event e.g. 平均每日的 session_duration 為 day

method: 底層 group by 計算的邏輯 e.g. 每個 view_event 'max' 的 measure_dwell_time ，其中 boolean, count_distinct 為輸入會特殊處理

event_type: tracking metric 的類型 可設可不設，會用是不是 view 開頭判斷

param: trigger 的 condition 要計算的 event -> 跟 trigger 同架構

condition: metric 的條件 e.g. measure_dwell_time 的是 view_item, shop->item 是 view_item:{from_screen:shop}


## metric1 - e1_e2

In [5]:
# metric 一次僅能算一個，主要由 metric_trigger 以及 metric_main 控制

metric1_input = {
    # 計算 metric 額外的 trigger
    'metric_trigger': {
        # 來源資料表
        'source': 'journey',
        # param
        'param': {
            'funnel_e1': None
        }
    },
    # 主要 metric 計算邏輯
    'metric_main': {
        # 自定 metric 名稱
        'metric_name': 'e1_e2',
        # 來源資料表
        'source': 'journey',
        # Group層級: user/day/event
        'group_level': 'user',
        # AGG邏輯: max/min/avg/sum/count/boolean/count_distinct
        'method': 'boolean',
        # (Optional) Event_Type: view_event/event_on_screen，預設會根據 metric 的 key 決定，不需要額外設定
        'event_type': 'view_event',
        # 指標計算
        'metric': {
            'funnel_e2': None # 從 journey 表直接拿 funnel_e2 欄位
        }
    }
}

metric1_sql = data_.get_metric_data(mode='sql', metric=metric1_input)
# print(metric1_sql)
metric1_data = data_.get_metric_data(mode='data', metric=metric1_input)
print(metric1_data.shape)
metric1_data.head()

(35357, 6)


,exp_id,var_id,beacon_uid,device,device_e1_e2,all_e1_e2
0,beq_for_you_abexp,3,fb_10158445847644003,iphone,1,1
1,beq_for_you_abexp,0,pp_aazxs62xv9wntrmxtdosksb7vgey,iphone,1,1
2,beq_for_you_abexp,2,gg_105401742793555245945,iphone,1,1
3,beq_for_you_abexp,102,fortyone-soap,iphone,0,0
4,beq_for_you_abexp,2,fb_970579972954007,iphone,1,1


In [12]:
exp_results(
    metric1_data,
    column='e1_e2',
    test_var=[1,2],
    test_type=0,
    merge_aa=False
)

/var/folders/cd/16hmf2b9703_yhx2s826739r0000gn/T/ipykernel_72940/3170237873.py:30: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  display(pd.concat([df_metric.iloc[:4].applymap(lambda x:'{:.1%}'.format(x)) , df_metric.iloc[4:]]).reset_index().rename(name_dict,axis =1).style.apply(style_row, axis=1))


,,e1_e2
0,var_A,78.0%
1,var_B,77.7%
2,mean_diff,-0.3%
3,mean_diff%,-0.4%
4,is_significant,False
5,p_value,0.597391


all


var_id
0       0.78633
1      0.780386
2      0.776932
3      0.772701
101    0.877778
102    0.671528
Name: all_e1_e2, dtype: Float64

## metric2 - session_duration_per_day

In [9]:
# method 用 sum ，method 的定義為 group by level 當層的 agg function
# session_duration 可能一天多筆 (多 device)，所以用 sum agg，後續的層級統一用 avg (會算到 per user)

metric2_input = {
    # 主要 metric 計算邏輯
    'metric_main': {
        # 自定 metric 名稱
        'metric_name': 'session_duration_per_day',
        # 來源資料表
        'source': 'journey',
        # Group層級: user/day/event
        'group_level': 'day',
        # AGG邏輯: max/min/avg/sum/count/boolean/count_distinct
        'method': 'sum',
        # 指標計算
        'metric': {
            'session_duration': None
        }
    }
}

metric2_sql = data_.get_metric_data(mode='sql', metric=metric2_input)
# print(metric2_sql)
metric2_data = data_.get_metric_data(mode='data', metric=metric2_input)
print(metric2_data.shape)
metric2_data.head()

(35413, 6)


,exp_id,var_id,beacon_uid,device,device_session_duration_per_day,all_session_duration_per_day
0,beq_for_you_abexp,1,ll_bl7nez5ds3wsj63sgs73yt2t5a,iphone,75.5,75.5
1,beq_for_you_abexp,2,gg_105852064054242284631,iphone,2884.0,2884.0
2,beq_for_you_abexp,1,fb_881295458594086,iphone,144.0,144.0
3,beq_for_you_abexp,3,fb_1989712147853199,iphone,620.0,620.0
4,beq_for_you_abexp,2,pp_abzo7u6chbmjq1uwq2_z6fignarx,iphone,8.0,8.0


## metric3 - session_duration_more_than_10min

In [14]:
# 如果 metric 是 journey ，可以直接在 metric 設置閾值算 boolean

metric3_input = {
    # 主要 metric 計算邏輯
    'metric_main': {
        # 自定 metric 名稱
        'metric_name': 'session_duration_more_than_10min',
        # 來源資料表
        'source': 'journey',
        # Group層級: user/day/event
        'group_level': 'day',
        # AGG邏輯: max/min/avg/sum/count/boolean/count_distinct
        'method': 'boolean',
        # 指標計算
        'metric': {
            'session_duration': {
                'cal':'>=',
                'value':'600'
            }
        }
    }
}

# metric3_sql = data_.get_metric_data(mode='sql', metric=metric3_input)
# print(metric3_sql)
metric3_data = data_.get_metric_data(mode='data', metric=metric3_input)
print(metric3_data.shape)
metric3_data.head()

(35413, 6)


,exp_id,var_id,beacon_uid,device,device_session_duration_more_than_10min,all_session_duration_more_than_10min
0,beq_for_you_abexp,2,gg_101657448999441083685,iphone,0.500000,0.500000
1,beq_for_you_abexp,0,fb_10154343759247418,android,0.000000,0.000000
2,beq_for_you_abexp,0,nicky711,iphone,0.461538,0.461538
3,beq_for_you_abexp,1,fb_10210909504960350,iphone,0.285714,0.285714
4,beq_for_you_abexp,3,pp_aafh7ckbodcuqjywsbsugifzhxuh,iphone,0.500000,0.500000


## metric4 - avg.view_feed 

In [11]:
metric4_input = {
    # 主要 metric 計算邏輯
    'metric_main': {
        # 自定 metric 名稱
        'metric_name': 'avg_view_feed',
        # 來源資料表
        'source': 'tracking',
        # Group層級: user/day/event
        'group_level': 'user',
        # AGG邏輯: max/min/avg/sum/count/boolean/count_distinct
        'method': 'count_distinct',
        # 指標計算
        'metric':{
            'view_feed':'view_id'
        }
    }
}

metric4_sql = data_.get_metric_data(mode='sql', metric=metric4_input)
# print(metric4_sql)
metric4_data = data_.get_metric_data(mode='data', metric=metric4_input)
print(metric4_data.shape)
metric4_data.head()

(35413, 6)


,exp_id,var_id,beacon_uid,device,device_avg_view_feed,all_avg_view_feed
0,beq_for_you_abexp,3,ll_7svtwpynxbno4drezkksoeiwty,iphone,1,1
1,beq_for_you_abexp,2,fb_3714905718590406,iphone,1,1
2,beq_for_you_abexp,1,wgarfie,iphone,6,6
3,beq_for_you_abexp,2,fb_10155020929023997,iphone,1,1
4,beq_for_you_abexp,0,pp_abzp9na3a-cpqagsdvzidzlhxaur,iphone,1,1


## metric5 - avg.dwell_time_from_feed_per_view 

In [12]:
metric5_input = {
    # 主要 metric 計算邏輯
    'metric_main': {
        # 自定 metric 名稱
        'metric_name': 'avg_dwell_time_from_feed_per_view',
        # 來源資料表
        'source': 'tracking',
        # Group層級: user/day/event
        'group_level': 'event',
        # AGG邏輯: max/min/avg/sum/count/boolean/count_distinct
        'method': 'max',
        # metric 的條件
        'condition':{
            'measure_dwell_time':{
                'screen_name':'feed'
            },
            'view_feed':None
        },
        # 指標計算
        'metric':{
            'measure_dwell_time':'current_timer'
        }
    }
}

metric5_sql = data_.get_metric_data(mode='sql', metric=metric5_input)
# print(metric5_sql)
metric5_data = data_.get_metric_data(mode='data', metric=metric5_input)
print(metric5_data.shape)
metric5_data.head()

(35413, 6)


,exp_id,var_id,beacon_uid,device,device_avg_dwell_time_from_feed_per_view,all_avg_dwell_time_from_feed_per_view
0,beq_for_you_abexp,2,gg_103625852848416749113,iphone,11.25,11.25
1,beq_for_you_abexp,3,nashi9,iphone,5.00,5.00
2,beq_for_you_abexp,1,happii,iphone,120.00,120.00
3,beq_for_you_abexp,1,we-buy-900,android,0.00,0.00
4,beq_for_you_abexp,3,jungvgh,iphone,6.00,6.00


## metric6 - avg.view_item_from_feed_per_view 

In [13]:
# 如果 group by event 並且 event 是 view_type 就一定要設置 event_trigger ，用來判斷 view ->view 的次數或成效

metric6_input = {
    # 計算 metric 額外的 trigger
    'metric_trigger':{
        # 來源資料表
        'source':'tracking',
        # Param
        'param':{
            'view_feed':None
        }
    },
    # 主要 metric 計算邏輯
    'metric_main': {
        # 自定 metric 名稱
        'metric_name': 'avg_view_item_from_feed_per_view',
        # 來源資料表
        'source': 'tracking',
        # Group層級: user/day/event
        'group_level': 'event',
        # AGG邏輯: max/min/avg/sum/count/boolean/count_distinct
        'method': 'count_distinct',
        # metric 的條件
        'condition':{
            'view_item':{
                'from_screen':'feed'
            }
        },
        # 指標計算
        'metric':{
            'view':'view_id'
        }
    }
}

metric6_sql = data_.get_metric_data(mode='sql', metric=metric6_input)
# print(metric6_sql)
metric6_data = data_.get_metric_data(mode='data', metric=metric6_input)
print(metric6_data.shape)
metric6_data.head()

(35334, 6)


,exp_id,var_id,beacon_uid,device,device_avg_view_item_from_feed_per_view,all_avg_view_item_from_feed_per_view
0,beq_for_you_abexp,1,ll_u3y22l43ec5nc5liwaio63hqnq,iphone,0.0,0.0
1,beq_for_you_abexp,3,olik,iphone,0.0,0.0
2,beq_for_you_abexp,2,fb_5947161781990840,android,0.0,0.0
3,beq_for_you_abexp,2,pp_abnrer9t7umhqlu8jgirqjlt5rgg,iphone,0.0,0.0
4,beq_for_you_abexp,2,vc7382,iphone,0.0,0.0


## metric7 - feed_to_item_adoption

In [5]:
# 也可以設置 boolean，判斷 view_feed->view_item 的 adoption
# 同樣需要設置 metric_trigger，並且 boolean 可以不用 input metric ，直接用 condition 判斷

metric7_input = {
    'metric_trigger':{
        'source':'tracking',
        'param':{
            'view_feed':None
        }
    },
    'metric_main':{
        'metric_name':'view_feed_view_item_adoption',
        'source':'tracking',
        'group_level':'event',
        'method':'boolean',
        'condition':{
            'view_item':{
                'from_screen':'feed'
                }
            }
    }
}

metric7_sql = data_.get_metric_data(mode='sql', metric=metric7_input)
print(metric7_sql)
# metric7_data = data_.get_metric_data(mode='data', metric=metric7_input)
# print(metric7_data.shape)
# metric7_data.head()

WITH expt AS
  (SELECT beacon,
          exp_id,
          var_id,
          date(created) AS created_date
   FROM ath_experiment
   WHERE exp_id = 'beq_for_you_abexp'
     AND date(created) <= date '2023-10-1' ),
     buyer_journey AS
  (SELECT visit_date,
          first_session_timestamp,
          date(at_timezone(first_session_timestamp, 'Asia/Taipei')) AS session_date,
          beacon_uid,
          b.beacon AS beacon,
          b.device AS device,
          b.session_duration_sec session_duration,
          geo,
          max_member_status,
          is_seller,
          first_session_timestamp,
          funnel_e1,
          funnel_e2,
          funnel_interest,
          funnel_consideration,
          add_to_cart,
          funnel_purchase,
          e2__uniq_items,
          if(funnel ['view_item'] IS NULL, 0, funnel ['view_item'].event_cnt) view_item_cnt,
          e2__session_duration,
          interest__shop_list,
          interest__item_list,
          search__term_li

In [16]:
# group by user 事例
metric8_input = {
    'metric_trigger':{
        'source':'tracking',
        'param':{
            'view_feed':None
        }
    },
    'metric_main':{
        'metric_name':'view_feed_to_item',
        'source':'tracking',
        'group_level':'user',
        'method':'boolean',
        'condition':{
            'view_item':{
                'from_screen':'feed'
            }
        }
    }
}

metric8_sql = data_.get_metric_data(mode='sql', metric=metric8_input)
# print(metric8_sql)
metric8_data = data_.get_metric_data(mode='data', metric=metric8_input)
print(metric8_data.shape)
metric8_data.head()


(35334, 6)


,exp_id,var_id,beacon_uid,device,device_view_feed_to_item,all_view_feed_to_item
0,beq_for_you_abexp,1,fb_1156576037700069,iphone,0,0
1,beq_for_you_abexp,1,gg_101724136907233074512,android,0,0
2,beq_for_you_abexp,0,fb_1913144932037147,iphone,0,0
3,beq_for_you_abexp,1,fb_962215013815921,android,0,0
4,beq_for_you_abexp,0,tana,iphone,0,0
